# Chess engine project - Tests and analysis

In this file I conduct various types of tests and analyses. 

The outcomes are used in the presentation in chess_engine/doc/.

## Contents

- [SimpleEngine tournament](#SimpleEngine-tournament)
- [Speed test evaluation function](#Speed-test-evaluation-function)
- [Analyze pruning and move-ordering](#Analyze-pruning-and-move-ordering)


In [1]:
# Change working directory to project root
import os
print(os.getcwd( )[-17:])
os.chdir('..') 
print(os.getcwd( )[-12:])

chess_engine\test
chess_engine


In [2]:
# Imports

import chess

import time

from src.chess_engine import tournament
from src.chess_engine import SimpleEngine, NegamaxEngine
from src.chess_engine.evaluation import *
from src.chess_engine.utils import make_board_numeric

## SimpleEngine tournament

Let three heuristics in the SimpleEngine class play against each other in a tournament.

In [4]:
# Define players
random_engine = SimpleEngine('random')
attacking_engine = SimpleEngine('attacking')
limiting_engine = SimpleEngine('limiting')
negamax_engine1 = NegamaxEngine(depth=1)
negamax_engine2 = NegamaxEngine(depth=2)
negamax_engine3 = NegamaxEngine(depth=3)
players = [random_engine, attacking_engine, limiting_engine, negamax_engine1, negamax_engine2, negamax_engine3]

# Run tournament
results = tournament(players, num_games=10)

Match: randomEngine vs randomEngine: [1, 7, 2, 0]
Match: randomEngine vs attackingEngine: [0, 4, 6, 0]
Match: randomEngine vs limitingEngine: [0, 8, 2, 0]
Match: randomEngine vs NegamaxEngine(depth=1): [0, 7, 3, 0]
Match: randomEngine vs NegamaxEngine(depth=2): [0, 5, 5, 0]
Match: randomEngine vs NegamaxEngine(depth=3): [0, 2, 8, 0]
Match: attackingEngine vs randomEngine: [4, 6, 0, 0]
Match: attackingEngine vs attackingEngine: [0, 10, 0, 0]
Match: attackingEngine vs limitingEngine: [0, 9, 1, 0]
Match: attackingEngine vs NegamaxEngine(depth=1): [1, 7, 2, 0]
Match: attackingEngine vs NegamaxEngine(depth=2): [0, 5, 5, 0]
Match: attackingEngine vs NegamaxEngine(depth=3): [0, 4, 6, 0]
Match: limitingEngine vs randomEngine: [4, 6, 0, 0]
Match: limitingEngine vs attackingEngine: [0, 10, 0, 0]
Match: limitingEngine vs limitingEngine: [0, 9, 1, 0]
Match: limitingEngine vs NegamaxEngine(depth=1): [4, 6, 0, 0]
Match: limitingEngine vs NegamaxEngine(depth=2): [0, 9, 1, 0]
Match: limitingEngine vs 

In [12]:
# Print Tournament results
print('TOURNAMENT RESULTS:\n')
for k, v in results.items():
    print(k, v)

# Winner is the NegamaxEngine (higher depth is btter)
print('\nWinner is the NegamaxEngine (higher depth is better).')

TOURNAMENT RESULTS:

randomEngine [3, 62, 55, 0]
attackingEngine [12, 81, 27, 0]
limitingEngine [14, 79, 27, 0]
NegamaxEngine(depth=1) [9, 103, 8, 0]
NegamaxEngine(depth=2) [33, 77, 10, 0]
NegamaxEngine(depth=3) [56, 64, 0, 0]

Winner is the NegamaxEngine (higher depth is better).


## Speed test evaluation function

In [3]:
# Number of evaluation calls
n = 100_000

# Instantiate board
board = chess.Board()
numeric_board = make_board_numeric(board)

# Initial numba call for setup
evaluation = request_evaluation(board)

In [4]:
print(f'Number of evaluations: {n}')

# Test speed of making board numeric
t0 = time.perf_counter()
for i in range(n):
    evaluation = make_board_numeric(board)
t1 = time.perf_counter()
make_numeric_time = t1-t0

# WITHOUT NUMBA
print('\nSPEED TESTS WITHOUT NUMBA')
print(f'\nmake_board_numeric(board): {round(make_numeric_time, 2)}s')

# Test speed of running the evaluation function
t0 = time.perf_counter()
for i in range(n):
    evaluation = compute_evaluation_nonumba(numeric_board)
t1 = time.perf_counter()
evaluation_time_nonumba = t1-t0
print(f'compute_evaluation_nonumba(numeric_board): {round(evaluation_time_nonumba, 2)}s')

# Test total evaluation time
t0 = time.perf_counter()
for i in range(n):
    evaluation = request_evaluation_nonumba(board)
t1 = time.perf_counter()
total_time_nonumba = t1-t0
print(f'request_evaluation_nonumba(board): {round(total_time_nonumba, 2)}s')

# WITH NUMBA
print('\nSPEED TESTS WITH NUMBA')
print(f'\nmake_board_numeric(board): {round(make_numeric_time, 2)}s')

# Test speed of running the evaluation function
t0 = time.perf_counter()
for i in range(n):
    evaluation = compute_evaluation(numeric_board)
t1 = time.perf_counter()
evaluation_time_numba = t1-t0
print(f'compute_evaluation_nonumba(numeric_board): {round(evaluation_time_numba, 2)}s')

# Test total evaluation time
t0 = time.perf_counter()
for i in range(n):
    evaluation = request_evaluation(board)
t1 = time.perf_counter()
total_time_numba = t1-t0
print(f'request_evaluation_nonumba(board): {round(total_time_numba, 2)}s')

# PERFORMANCE IMPROVEMENT
print('\nPERFORMANCE IMPROVEMENT')
print('\nEvaluation time improvement: {improvement}%'.format(
    improvement = round(100 * (1 - evaluation_time_numba / evaluation_time_nonumba), 2)))
print('Total time improvement: {improvement}%'.format(
    improvement = round(100 * (1 - total_time_numba / total_time_nonumba), 2)))

Number of evaluations: 100000

SPEED TESTS WITHOUT NUMBA

make_board_numeric(board): 4.22s
compute_evaluation_nonumba(numeric_board): 4.06s
request_evaluation_nonumba(board): 9.18s

SPEED TESTS WITH NUMBA

make_board_numeric(board): 4.22s
compute_evaluation_nonumba(numeric_board): 0.36s
request_evaluation_nonumba(board): 4.4s

PERFORMANCE IMPROVEMENT

Evaluation time improvement: 91.13%
Total time improvement: 52.05%


## Analyze pruning and move-ordering

In this section, I analyze the reduction of the necessary number evaluations for alpha-beta pruning with different mthods of move-ordering.

In [5]:
# Import previous versions of NegamaxEngine with less features for comparison
from src.chess_engine.engine_versions import *

In [6]:
def compare_versions(depth, board):
    # player1 = MinimaxEngine(depth=depth)
    player2 = NegamaxEngineV1(depth=depth) # no pruning
    # player3 = NegamaxEngineV2(depth=depth)
    # player4 = NegamaxEngineV3(depth=depth)
    # player5 = NegamaxEngineV4(depth=depth)
    player6 = NegamaxEngineV5(depth=depth) # ab pruning, but no move ordering
    player7 = NegamaxEngineV6(depth=depth) # ab pruning, random move ordering
    player8 = NegamaxEngineV7(depth=depth) # ab pruning, systematic move ordering
    player_list = [player2, player6, player7, player8]
    
    for player in player_list:
        t0 = time.perf_counter()
        move = player.make_move(board)
        t1 = time.perf_counter()
        print('{player} | time: {time}s | num_evals: {num_evals} | move: {move}'.format(
            player=str(player), time=round(t1-t0, 4), num_evals=player.num_evals, move=move))

In [9]:
# exemplary position
board=chess.Board('r3k2r/pbp1bp2/2nqpp2/1p1p3p/P3P1QP/1PNP1N2/2P1BPP1/R3K2R b KQkq - 0 11')

print('\nDepth=1')
compare_versions(depth=1, board=board)

print('\nDepth=2')
compare_versions(depth=2, board=board)

print('\nDepth=3')
compare_versions(depth=3, board=board)

print('\nDepth=4')
compare_versions(depth=4, board=board)


Depth=1
NegamaxEngineV1(depth=1,no_pruning) | time: 0.0071s | num_evals: 39 | move: h5g4
NegamaxEngineV5(depth=1,ab_pruning,no_ordering) | time: 0.0073s | num_evals: 39 | move: h5g4
NegamaxEngineV6(depth=1,ab_pruning,random_ordering) | time: 0.0072s | num_evals: 39 | move: h5g4
NegamaxEngineV7(depth=1,ab_pruning,systematic_ordering) | time: 0.0078s | num_evals: 39 | move: h5g4

Depth=2
NegamaxEngineV1(depth=2,no_pruning) | time: 0.2348s | num_evals: 1689 | move: h5g4
NegamaxEngineV5(depth=2,ab_pruning,no_ordering) | time: 0.0288s | num_evals: 243 | move: h5g4
NegamaxEngineV6(depth=2,ab_pruning,random_ordering) | time: 0.032s | num_evals: 161 | move: h5g4
NegamaxEngineV7(depth=2,ab_pruning,systematic_ordering) | time: 0.0307s | num_evals: 117 | move: h5g4

Depth=3
NegamaxEngineV1(depth=3,no_pruning) | time: 7.5558s | num_evals: 63455 | move: h5g4
NegamaxEngineV5(depth=3,ab_pruning,no_ordering) | time: 0.7614s | num_evals: 5927 | move: h5g4
NegamaxEngineV6(depth=3,ab_pruning,random_orde